In [ ]:
%reload_ext autoreload
%autoreload 2 
import numpy as np
import matplotlib.pyplot as plt  

from env import SingleStateSpace 
from env import DoubleStateSpace
from RewardFunctions import SingleStateReward
from RewardFunctions import DoubleStateReward

from policy_walk import PolicyWalk 
from priors import UniformPrior 
from priors import TimeDependentPrior
from priors import SkewSymmetricGaussianPrior

from generate_trajectory import generate_trajectory
from policy import Boltzmann 
from learn import policy_iteration

import tensorflow_probability as tfp 

import os 
import cProfile
import re 
# cProfile.run('re.compile(pw.get_samples(step_size, n_iters = 1000))')

In [ ]:
#Set up the environment
# trajectory_length = 10
# env = SingleStateSpace(n_actions = 10, discount_rate = 0.1, R_max = 5)

# #generate observation times 
# # observation_times = np.cumsum(np.random.uniform(0, 2, size=trajectory_length))
# observation_times = [1.85,2.54,4.13,5.09,7.02,7.89,9.86,11.82,12.03,12.87]

# # Create a matrix for the true rewards 
# s_grid, a_grid, t_grid = np.meshgrid(env.states, env.actions, observation_times, indexing='ij')
# R = np.vectorize(env.reward)(s_grid,a_grid,t_grid)

# #Find the optimal policy, values and q_values for the true reward to generate the trajectory. 
# alpha = 2
# (optimal_pi, optimal_values, optimal_q_values) = policy_iteration(env,trajectory_length,R)
# b = Boltzmann(optimal_q_values, env.actions, alpha = alpha)

# #multiple trajectories are combined - just means that the agent does multiple things at each time slot to allow for more data to be gathered
# observations = generate_trajectory(env,b,observation_times, n_trajectories=50) 

# # mcmc_steps = ["grid_walk", "gaussian_scalar"]
# # priors = ["ssgp", "tdp single", "tdp full"]
# mcmc_steps = ["gaussian_scalar"]
# priors = ["ssgp", "tdp single"]
# for mcmc_step in mcmc_steps: 
#     for p in priors: 
#         # if mcmc_step == "gaussian_scalar" and p != "tdp full": 
#         #     continue 
        
#         # else:
#         aps = [] 
#         lrs = [] 
#         # sigmas = [0.5,1,1.5]
#         # step_sizes = [0.01, 0.05, 0.1]
#         sigmas = [0.01,0.1,0.5,1,1.5,2]
#         step_sizes = [0.005,0.01,0.05,0.1,0.5,1]
#         if p == "ssgp": 
#             sigmas = [None] 
#         for step_size in step_sizes: 
#             for sigma in sigmas: 
#                 if p == "ssgp": 
#                     prior = SkewSymmetricGaussianPrior()
#                 elif p == "tdp single": 
#                     prior = TimeDependentPrior(observation_times, R_max = 5, sigma = sigma, full = False) 
#                 elif p == "tdp full": 
#                     prior = TimeDependentPrior(observation_times, R_max = 5, sigma = sigma, full = True)

#                 pw = PolicyWalk(env,prior,observations,observation_times,alpha,mcmc_step = mcmc_step)
#                 learned_rewards, acceptance_probs = pw.get_samples(step_size, n_iters = 100000)
#                 aps.append(acceptance_probs) 
#                 lrs.append(learned_rewards)
#         name = p + " " + mcmc_step
#         folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc = save(True, name)
#         doAcceptanceAnalysis(AP_folder_loc)
#         doChainAnalysis(chain_folder_loc)
#         doLearnedRewardAnalysis(0,2, LR_folder_loc)
#         doLossAnalysis(folderloc)

trajectory_length = 10
# env = DoubleStateSpace(n_actions = 10, discount_rate = 0.1, R_max = 5)

# #generate observation times 
# # observation_times = np.cumsum(np.random.uniform(0, 2, size=trajectory_length))
# observation_times = np.load("Double State Experiments/observation_times.npy")

# # Create a matrix for the true rewards 
# s_grid, a_grid, t_grid = np.meshgrid(env.states, env.actions, observation_times, indexing='ij')
# R = np.vectorize(env.reward)(s_grid,a_grid,t_grid)

# #Find the optimal policy, values and q_values for the true reward to generate the trajectory. 
# alpha = 2
# (optimal_pi, optimal_values, optimal_q_values) = policy_iteration(env,trajectory_length,R)
# b = Boltzmann(optimal_q_values, env.actions, alpha = alpha)

# #multiple trajectories are combined - just means that the agent does multiple things at each time slot to allow for more data to be gathered
# # observations = generate_trajectory(env,b,observation_times, n_trajectories=50) 

# observations = np.load("Double State Experiments/observations.npy")


# mcmc_steps = ["gaussian_scalar"]
# priors = ["ssgp", "tdp single"]
mcmc_steps = ["gaussian_scalar","grid_walk"]
priors = ["ssgp", "tdp single", "tdp full"]
for mcmc_step in mcmc_steps: 
    for p in priors: 
        # if mcmc_step == "gaussian_scalar" and p != "tdp full": 
        #     continue 
        # else:
        aps = [] 
        lrs = [] 
        sigmas = [0.5,1,1.5]
        step_sizes = [0.01, 0.05, 0.1]
        if p == "ssgp": 
            sigmas = [None] 
        for step_size in step_sizes: 
            for sigma in sigmas: 
                if p == "ssgp": 
                    prior = SkewSymmetricGaussianPrior()
                elif p == "tdp single": 
                    prior = TimeDependentPrior(observation_times, R_max = 5, sigma = sigma, full = False) 
                elif p == "tdp full": 
                    prior = TimeDependentPrior(observation_times, R_max = 5, sigma = sigma, full = True)

                pw = PolicyWalk(env,prior,observations,observation_times,alpha,mcmc_step = mcmc_step)
                learned_rewards, acceptance_probs = pw.get_samples(step_size, n_iters = 100000)
                aps.append(acceptance_probs) 
                lrs.append(learned_rewards)
        
        name = p + " " + mcmc_step
        folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc = save(False, name)
        doAcceptanceAnalysis(AP_folder_loc)
        doChainAnalysis(chain_folder_loc)
        doLearnedRewardAnalysis(0,2, LR_folder_loc)
        doLearnedRewardAnalysis(1,2, LR_folder_loc)
        doLossAnalysis(folderloc)

# # policy_walk_iters = 1
# # for _ in range(policy_walk_iters-1): 
# #     learned_rewards = np.concatenate((learned_rewards, pw.get_samples(step_size, n_iters = 150000)))

In [ ]:
name = p + " " + mcmc_step
step_sizes = [0.005,0.01,0.05,0.1,0.5]
folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc = save(True, name)
doAcceptanceAnalysis(AP_folder_loc)
doChainAnalysis(chain_folder_loc)
doLearnedRewardAnalysis(0,2, LR_folder_loc)
doLossAnalysis(folderloc)

In [ ]:
def get_folders(single, title): 
    if single: 
        folderloc = "Single State Experiments/" + title +"/"
    else:
        folderloc = "Double State Experiments/" + title + "/"
    AP_folder_loc = folderloc + "/Screenshots/Acceptance Probabilities/"
    LR_folder_loc = folderloc + "/Screenshots/Rewards/"
    chain_folder_loc = folderloc + "/Screenshots/Chain/"
    return folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc

In [ ]:
#Folder handling for saving experiment results 

def save(single, title):
    if single: 
        folderloc = "Single State Experiments/" + title +"/"
    else:
        folderloc = "Double State Experiments/" + title + "/"
    AP_folder_loc = folderloc + "/Screenshots/Acceptance Probabilities/"
    LR_folder_loc = folderloc + "/Screenshots/Rewards/"
    chain_folder_loc = folderloc + "/Screenshots/Chain/"
    if not os.path.isdir(folderloc): 
        os.mkdir(folderloc) 
    if not os.path.isdir(folderloc + "/Screenshots/"):    
        os.mkdir(folderloc + "/Screenshots/")
    if not os.path.isdir(AP_folder_loc): 
        os.mkdir(AP_folder_loc)
    if not os.path.isdir(LR_folder_loc): 
        os.mkdir(LR_folder_loc)
    if not os.path.isdir(chain_folder_loc): 
        os.mkdir(chain_folder_loc)
    np.save(folderloc + "aps", aps) 
    np.save(folderloc + "lrs", lrs) 
    f = open(folderloc + "info.txt", "w+") 
    f.write("Sigmas: " + format(sigmas) +"\r\n")
    f.write("Step Sizes: " + format(step_sizes) + "\r\n") 
    f.write("Prior: " + format(prior)+ "\r\n") 
    f.write("Jumping Distribution: " + mcmc_step + "\r\n")
    f.write("Alpha: " + format(alpha)) 
    f.close()
    return folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc

In [ ]:
import matplotlib.pyplot as plt 
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size":    "16.0",
    "figure.figsize" : "8, 5",
    "axes.titlesize": "medium",
    "figure.titlesize": "medium",
    "savefig.dpi" : "250",
    "savefig.format" : "png",
    "savefig.bbox": "tight"
})

Analysis of Acceptance Probabilities:

In [ ]:
#Scatter plot of thinned acceptance probabilities - change policy_walk to return acceptance probabilities to use
# print(np.average(acceptance_probs))
# plt.scatter(range(len(acceptance_probs[14::15])), acceptance_probs[14::15])
# plt.show()
import math
# Moving average of acceptance probabilities
def doAcceptanceAnalysis(folder):
    i = 0 
    for step_size in step_sizes: 
        for sigma in sigmas: 
            acceptance_probs = aps[i]
            window_size = 500
            n_windows = math.floor(len(acceptance_probs)/window_size) 
            averages = [] 
            for j in range(n_windows): 
                averages.append(np.average(acceptance_probs[window_size*i:window_size*(j+1)]))
            plt.plot(np.arange(0, n_windows * window_size, window_size), averages, label = "Step: " + format(step_size))
            plt.xlabel("Sample Number")
            plt.ylabel("Avg Acceptance Probability")
            if sigma is None: 
                plt.title("Acceptance Probabilities (Step: " + format(step_size) +")")
            elif step_size is None: 
                plt.title("Acceptance Probabilities (sigma: " + format(sigma)+ ")")
            else: 
                plt.title("Acceptance Probabilities (Step: " + format(step_size) +", sigma: " + format(sigma)+ ")")
            name = "step " + format(step_size) + " sigma " + format(sigma) + ".png"
            i+=1 
    # name = "all step sizes.png"
            if sigma is not None: 

                plt.savefig(folder + name, format = "png")
                plt.show() 
    if sigma is None: 
        plt.legend(loc = "upper right")
        plt.savefig(folder + "all step sizes.png", format = "png") 
        plt.show()



Analysis of Average Learned Reward Matrix

In [ ]:
#Shows the overall learned rewards - switch to the bigger colors array to view all 
colors = ["#e71d43","#ff0000","#ff3700","#ff6e00","#ffa500","#ffc300","#ffe100","#ffff00","#aad500","#55aa00","#008000","#005555","#002baa","#0000ff","#1900d5","#3200ac","#4b0082","#812ba6","#b857ca","#d03a87"]
# colors = ["red", "orange","yellow","green","blue","indigo","violet"]

# s,a,T = 0,0,10 
T = 10
def doLearnedRewardAnalysis(s,a, folder):
    i = 0 
    for step_size in step_sizes: 
        for sigma in sigmas: 
            lr = lrs[i]
            avg_reward = np.average(lr[50000:], axis = 0)
            for t in range(T):
                label = "t = " + format(round(observation_times[t],2))
                plt.plot(avg_reward[s,:,t], color = colors[t*2], label = label)
            plt.ylabel("Reward")
            plt.xlabel("Action")
            
            if sigma is None: 
                plt.title("Learned Reward (Step: " + format(step_size) +")")
            elif step_size is None: 
                plt.title("Learned Reward (sigma: " + format(sigma)+ ")")
            else: 
                plt.title("Learned Reward (Step: " + format(step_size) +", sigma: " + format(sigma)+ ")")
            plt.legend(loc = "upper right")
            if s == 0: 
                name = "step " + format(step_size) + " sigma " + format(sigma) + ".png"
            else:
                name = "Hungry step " + format(step_size) + " sigma " + format(sigma) + ".png"
            plt.savefig(folder + name, format = "png") 
            plt.show()
            i+=1

    for t in range(T):
        label = "t = " + format(round(observation_times[t],2))
        plt.plot(R[s,:,t], color=colors[t*2], label = label)
    plt.ylabel("Reward")
    plt.xlabel("Actions")
    plt.title("True Reward") 
    plt.legend(loc = "upper right")
    plt.savefig(folder + "True Reward.png", format = "png") 
    plt.show()


#Plotting samples for different sigma values. 
# for sigma in [0.001, 0.01, 0.1, 1, 2, 5, 100, 1000]:
    
    # prior = TimeDependentPrior(observation_times, R_max = 5, sigma = sigma)
    # prior_sample = prior.sample(env.n_states, env.n_actions, trajectory_length)
    # for t in range(T):
    #     label = "t = " + format(round(observation_times[t],2))
    #     plt.plot(prior_sample[s,:,t], color=colors[t*2], label = label)
    # # for a in [0,4]:
    # #     plt.plot(observation_times, prior_sample[s,a,:], label = "a = " + format(a))
    # plt.ylabel("Reward")
    # # plt.xlabel("Time")
    # plt.xlabel("Actions")
    # plt.legend()
    # plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def l2loss(reward, true_reward): 
    return np.linalg.norm((reward - true_reward).flatten(), 2) 

def doLossAnalysis(folderloc):
    losses = []
    xaxis = []

    prev_sigma = None  # Variable to track the previous sigma value
    i = 0
    f = open(folderloc + "losses.txt", "w+") 
    for step_size in step_sizes:
        for sigma in sigmas: 
            lr = lrs[i] 
            loss = l2loss(np.average(lr[50000:], axis = 0), R)
            f.write(format(loss) + "\n")
            losses.append(loss)
            xaxis.append((step_size, sigma))
            i += 1
    if sigma is not None: 
        plt.xlabel("Sigma")
    plt.ylabel("L2 Loss")
    plt.title("L2 Losses of Learned Rewards for Different Hyperparameters")

    # Set up step size colors and legend
    step_size_colors = plt.cm.get_cmap("rainbow", len(step_sizes))
    step_size_labels = [f"Step Size: {step_size}" for step_size in step_sizes]
    step_size_patches = [plt.Rectangle((0, 0), 1, 1, color=step_size_colors(i)) for i in range(len(step_sizes))]
    plt.legend(step_size_patches, step_size_labels, loc="lower right")

    # Create the bar chart
    bar_width = 3 / len(sigmas)  # Width of each bar
    x_positions = np.arange(len(xaxis))
    xtick_positions = []  # Positions for x-axis tick labels
    xtick_labels = []  # Labels for x-axis tick labels

    for i, (step_size, sigma) in enumerate(xaxis):
        if sigma != prev_sigma:  # Check if sigma has changed
            xtick_positions.append(x_positions[i])
            xtick_labels.append(sigma)
            prev_sigma = sigma

        plt.bar(x_positions[i], losses[i], width=bar_width, color=step_size_colors(step_sizes.index(step_size)))

    plt.plot([np.min(losses)]*len(sigmas)*len(step_sizes), color = 'black', linestyle = ':', label = 'Min: ' + format(np.min(losses)))

    # Adjust x-axis ticks and labels
    plt.xticks(xtick_positions, xtick_labels, rotation='vertical')
    name = "L2 Losses of Learned Rewards for Different Hyperparameters.png"
    plt.savefig(folderloc + "/Screenshots/" + name, format = "png", ) 
    plt.show()
    f.close()

Spotify Analysis

In [ ]:
colors = ["#e71d43","#ff0000","#ff3700","#ff6e00","#ffa500","#ffc300","#ffe100","#ffff00","#aad500","#55aa00","#008000","#005555","#002baa","#0000ff","#1900d5","#3200ac","#4b0082","#812ba6","#b857ca","#d03a87"]
avg_reward = np.average(np.asarray(learned_rewards[200000:]),axis = 0)
# print(observation_times)
actions = ['Sea Girls', 'Christmas', 'Mature Cheddar', 'Renault Megane']
import pandas as pd 
date_range = pd.date_range(start='2021-12-09', periods = len(observation_times), freq = 'D')
# date_range = pd.date_range(start='2021-12-09', end='2023-04-28', freq='D')
highlight = 3
for a in range(0,4):
    alpha = 0.25 
    if a == highlight:
        alpha = 1 
    plt.plot(date_range, avg_reward[s,a,:], label = actions[a], alpha = alpha)

plt.xlabel("Date")
plt.ylabel("Reward")
plt.legend()
plt.show()

Chain Analysis

In [ ]:
#Used to analyse individual chains: 
#Plot a single reward
s,a,t = 0,0,1
plt.plot(lrs[2][:,s,a,t])
ylabel = "Reward of (s,a,t) = (" + format(s) + "," + format(a) + "," + format(t) + ")"
plt.ylabel(ylabel)
plt.xlabel("Action")
plt.show()

Effective Sample Size Calculation:

In [ ]:
ess = tfp.mcmc.effective_sample_size(lrs[4][20000:])
print(np.average(ess))

In [ ]:
#Plot rewards over time for a given state/action pair - shows observed (s,a,t) pairs in colour 
# print(observations)

#I could try plotting like true reward vs assigned reward over time or something to better visualise idk oh maybe do colours based on reward 
def doChainAnalysis(folder):
    i = 0 
    colors = ["#e71d43","#ff0000","#ff3700","#ff6e00","#ffa500","#ffc300","#ffe100","#ffff00","#aad500","#55aa00","#008000","#005555","#002baa","#0000ff","#1900d5","#3200ac","#4b0082","#812ba6","#b857ca","#d03a87"]
    for step_size in step_sizes: 
        for sigma in sigmas: 
            lr = lrs[i]
            s,a, = 0,2
            # s,t = 0,5
            for t in range(trajectory_length):
                label = "t = " + format(round(observation_times[t],1)) + " (" + format(round(R[s,a,t],1)) + ")"
                if R[s,a,t] > 0.25:
                    plt.plot(lr[:,s,a,t], color=colors[t*2], label = label)
                else:
                    plt.plot(lr[:,s,a,t], color="grey", label = label)        
                
            plt.ylabel("R(" + format(s) + "," + format(a) + ",t)")
            plt.xlabel("Iteration")
            #don't need to do the sigma here, just copy the none code over from the other bit and do the same thing here.
            if sigma is None: 
                plt.title("Reward throughout Sampling (Step: " + format(step_size) +")")
            elif step_size is None: 
                plt.title("Reward throughout Sampling (sigma: " + format(sigma)+ ")")
            else: 
                plt.title("Reward throughout Sampling (Step: " + format(step_size) +", sigma: " + format(sigma)+ ")")
            plt.legend()
            name = "step " + format(step_size) + " sigma " + format(sigma) + ".png"
            plt.savefig(folder + name, format = "png") 
            plt.show()
            i+=1


Histogram Plots:

In [ ]:
converged_rewards = learned_rewards[37500:]
np.swapaxes(converged_rewards,1,2)
converged_rewards = np.reshape(converged_rewards, (converged_rewards.shape[0], converged_rewards.shape[1]*converged_rewards.shape[2], converged_rewards.shape[3]))
print(converged_rewards.shape)
print(observation_times)
s,a,t = 0,2,0
for t in range(20):
    plt.hist(converged_rewards[:,s,t],bins = 20,color=colors[t])
    plt.show()

Covariance Calculations

In [30]:
folderloc

'Double State Experiments/tdp full grid_walk/'

In [70]:
# learned_rewards.shape
# #Flatten the rewards 
best_lrs = np.load("Double State Experiments/tdp single gaussian_scalar/lrs.npy")
best_lr = best_lrs[4] 
# best_learned = np.average(best_lr[50000:], axis = 0) 
# print(l2loss(best_learned,R))

converged_rewards = best_lr[50000:]
flattened_rewards = converged_rewards.reshape(converged_rewards.shape[0], -1)
cov = np.cov(flattened_rewards.T)
# from statsmodels.stats.correlation_tools import cov_nearest
# pd_cov = cov_nearest(cov, threshold = 1e-8)

np.save("Double State Experiments/best_cov.npy", cov)

# zero_mean = np.zeros(flattened_rewards.shape[1])
# test = np.random.normal(np.zeros(flattened_rewards.shape[1]))
# print(pd_cov)
# np.save(folderloc,cov)
# print(cov)
# np.save('learned_rewards_short_spotify',learned_rewards)
# np.save('acceptance_probs_short_spotify', acceptance_probs)
# learned_rewards = np.load('learned_rewards_full_spotify.npy')
# acceptance_probs = np.load('acceptance_probs_full_spotify.npy')

In [ ]:
#For ReDoing Graphs 
trajectory_length = 10
env = DoubleStateSpace(n_actions = 10, discount_rate = 0.1, R_max = 5)

# #generate observation times 
observation_times = np.load("Double State Experiments/observation_times.npy")

# # Create a matrix for the true rewards 
s_grid, a_grid, t_grid = np.meshgrid(env.states, env.actions, observation_times, indexing='ij')
R = np.vectorize(env.reward)(s_grid,a_grid,t_grid)

observations = np.load("Double State Experiments/observations.npy")

mcmc_steps = ["gaussian_scalar","grid_walk"]
priors = ["ssgp", "tdp single", "tdp full"]
for mcmc_step in mcmc_steps: 
    for p in priors: 
        aps = [] 
        lrs = [] 
        sigmas = [0.5,1,1.5]
        step_sizes = [0.01, 0.05, 0.1]
        if p == "ssgp": 
            sigmas = [None] 
        
        name = p + " " + mcmc_step
        folderloc, AP_folder_loc, LR_folder_loc, chain_folder_loc = get_folders(False, name)
        lrs = np.load(folderloc + "lrs.npy") 
        aps = np.load(folderloc + "aps.npy")
        doAcceptanceAnalysis(AP_folder_loc)
        doChainAnalysis(chain_folder_loc)
        doLearnedRewardAnalysis(0,2, LR_folder_loc)
        doLearnedRewardAnalysis(1,2, LR_folder_loc)
        doLossAnalysis(folderloc)